In [9]:
#!pip install transformers
#!pip install huggingface_hub
#!pip install datasets

     -------------------------------------- 474.6/474.6 kB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 132.9/132.9 kB ? eta 0:00:00
     --------------------------------------- 21.5/21.5 MB 19.9 MB/s eta 0:00:00
     ---------------------------------------- 323.6/323.6 kB ? eta 0:00:00
     ---------------------------------------- 61.7/61.7 kB ? eta 0:00:00
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4


In [10]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
datasets = load_dataset("piqa")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset piqa downloaded and prepared to C:/Users/shwet/.cache/huggingface/datasets/piqa/plain_text/1.1.0/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def create_df(dataset):
    
    df = pd.DataFrame(dataset)
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    
    return df

In [28]:
datasets.keys()

dict_keys(['train', 'test', 'validation'])

In [60]:
datasets["train"][0]["goal"]

"When boiling butter, when it's ready, you can"

In [63]:
def preprocess_intents_json(segment):
    
    preprocessed_data = []
    
    for data in datasets[segment]:
        preprocessed_data.append(f"Goal: {data['goal']}\n")
        if data['label']=='1':
            preprocessed_data.append(f"Solution: {data['sol2']}\n")
        else:
            preprocessed_data.append(f"Solution: {data['sol1']}\n")
    
    return "".join(preprocessed_data)

def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)


output_file = f"./data/data_train.txt"
preprocessed_data = preprocess_intents_json("train")
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_validation.txt"
preprocessed_data = preprocess_intents_json("validation")
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_test.txt"
preprocessed_data = preprocess_intents_json("test")
save_preprocessed_data(preprocessed_data, output_file)

In [41]:
df_train = create_df(datasets["train"])
df_test = create_df(datasets["test"])
df_validation = create_df(datasets["validation"])

In [45]:
def select_correct_answer(df):
    df["sol"] = ""

    for i in range(len(df)):
        if df["label"][i]=="1":
            df["sol"][i] = df["sol2"][i]
        if df["label"][i]=="0":
            df["sol"][i] = df["sol1"][i]

    return df[["goal", "sol"]]


In [46]:
df_train = select_correct_answer(df_train)
df_test = select_correct_answer(df_test)
df_validation = select_correct_answer(df_validation)

In [48]:
display(df_train)
display(df_test)
display(df_validation)

,goal,sol
0,"When boiling butter, when it's ready, you can",Pour it into a jar
1,"To permanently attach metal legs to a chair, y...",Weld the metal together to get it to stay firm...
2,how do you indent something?,leave a space before starting the writing
3,how do you shake something?,move it up and down and side to side quickly.
4,Clean tires,"Pour water, scrape off caked on dirt. Use a st..."
...,...,...
16108,tack,can be use to secure broom to the wall
16109,How to clean a dishwasher.,Place a dishwasher safe bowl on the top rack o...
16110,"To seal leather for furniture,","place the leather on a work surface, pour a sm..."
16111,Make lemonade,Fill a glass about 3/4 full with water. Using ...


,goal,sol
0,how do you puncture a vein?,pop it.
1,hands,is used to put on milk jug
2,What ingredients do I need to make a shortcrus...,"To make pie crust, you will need flour, sugar,..."
3,roast broccoli,Preheat oven to 450 degrees F. Toss the bro...
4,To crimp the edges of the patsy crust.,Use a fork to crimp the edges
...,...,...
3079,What can I use to make my walkway look better?,Put small rocks on either side of the walkway ...
3080,how do you put pressure on something?,push down on it.
3081,Create tent bug screen.,Hang poncho on top of tent.
3082,How to fertilize an indoor plant?,Using egg shells on to the pot would fertilize...


,goal,sol
0,How do I ready a guinea pig cage for it's new ...,Provide the guinea pig with a cage full of a f...
1,dresser,"finish, woodgrain with bobby pin"
2,To fight Ivan Drago in Rocky for sega master s...,You have to defeat Apollo Creed and Clubber La...
3,Make outdoor pillow.,Blow into trash bag and tie with rubber band.
4,ice box,will turn into a cooler if you add water to it
...,...,...
1833,To add chocolate and caramel to hot milk and c...,Chop the chocolate into small chunks. Bring th...
1834,To make orange pulp from oranges.,Peel the oranges and put in blender.
1835,Roast marshmallows.,Create a fire using wood. Poke a marshmallow w...
1836,A power washer,can clean an aging cruise liner clear of grime


## Preprocessing Data

In [49]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, Trainer, TrainingArguments
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

model = GPT2Model.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

In [ ]:
"C:\Users\shwet\Downloads\archive (2)\intents.json"